In [ ]:
import os
import numpy as np
import pandas as pd

# Absolute path to current working directory
home = os.getcwd()

# Data pre-processing
Pipeline for pre-processing the data. This needed to be done in parts because of the big file sizes. 

In [ ]:
# For 10oct Tuesday data
path_to_dir = "/media/saaville/ARSETUS/data_catalonia/"
data = pd.read_json(path_to_dir + "10octs3.json", lines=True, orient="records", encoding="utf-8")

In [ ]:
# For 8oct Tuesday data
data = pd.read_json(home + "/catalonia/8oct_tweets.json", lines=True, orient="records", encoding="utf-8")

In [ ]:
data.shape

In [ ]:
data.dtypes

## String/Int issue with 'id_str'-column

Pandas is automatically casting all of the new __id_str__ to integers. This could make reliable comparison between strings difficult and might complicate the process of removing duplicates. We can address this by casting all 'id_str' values to str:

In [ ]:
# Ensure that id's are also in string in order to avoid problems with possible overflow of integers 
data['id_str'] = [str(tweet_id) for tweet_id in data['id_str']]

## Restructuring the dataset
Data is not yet easily accessible for analysis. Thus, next we are going to restructure the data by creating new columns from the three nested columns which are now inhabitated by multi-variable-dictionaries: __'place'__, __'user'__ and __'entities'__. The heavy-lifting is done using Python's list comprehensions:

In [ ]:
# Restructure important categories inside 'user', 'place' and 'entities'
data['user_id']               = [row['id'] for row in data['user']]
data['user_name']             = [row['name'] for row in data['user']]
data['user_screen_name']      = [row['screen_name'] for row in data['user']]
data['user_defined_location'] = [row['location'] if pd.notnull(row['location']) else NaN for row in data['user']]
data['user_followers_count']  = [row['followers_count'] for row in data['user']]
data['place_country_code']    = [row['country_code'] if row != None else None for row in data['place']]
data['place_name']            = [row['name'] if row != None else None for row in data['place']]
data['place_type']            = [row['place_type'] if row != None else None for row in data['place']]
data['place_coordinates']     = [dict(row['bounding_box'])['coordinates'][0][0] if row != None else None for row in data['place']]
data['hashtags']              = [[htags['text'].lower() for htags in row['hashtags']] for row in data['entities'] if row['hashtags'] != ""]
data['user_created_at']       = [row['created_at'] for row in data['user']]
data['user_geo_enabled']      = [row['geo_enabled'] for row in data['user']]
data['is_retweet']            = [1 if b else 0 for b in data['retweeted_status'].notnull()]

In [ ]:
# For retweets: extract the parent from column 'retweeted_status' 
parent_tweet_ids = []
for index, row in data.iterrows():
    if row['is_retweet']:
        tweet = row['retweeted_status']
        parent_tweet_ids.append(tweet['id_str'])
    else:
        parent_tweet_ids.append(None)

In [ ]:
data['parent_tweet_id'] = parent_tweet_ids

### Removing (or dropping) unnecessary columns
The processing of such large datasets can be speeded up by dropping/removing unnecessary columns, which we will do next:

In [ ]:
data.drop(['contributors','entities','extended_entities','geo',
           'user','place','in_reply_to_screen_name','in_reply_to_user_id',
           'in_reply_to_user_id_str','metadata','quoted_status_id_str',
           'in_reply_to_status_id','quoted_status','truncated','retweeted_status',
           'quoted_status_id','is_quote_status','in_reply_to_status_id_str'],
           axis=1, inplace=True)
data.dtypes

In [ ]:
data.shape

# Anomalies in the dataset
Detecting duplicates in columns __id_str__ and __id__.

In [ ]:
print("Duplicates in column id_str:", data['id_str'].duplicated().sum(), "\n" +
      "Duplicates in column id:", data['id'].duplicated().sum())

In [ ]:
duplicates_id_str = list(set(data[data['id_str'].duplicated()]['id_str']))

In [ ]:
# Let's take a closer look at these non-unique id_str-values
data[(data['id_str'] == duplicates_id_str[2]) | (data['id_str'] == duplicates_id_str[3]) | (data['id_str'] == duplicates_id_str[4]) | (data['id_str'] == duplicates_id_str[5])][['created_at','id','id_str','user_id','text','place_country_code']]

This is strange, Twitter states following regarding __id_str__:
>*The string representation of the unique identifier for this Tweet. Implementations should use this rather than the large integer in id*

It seems that id_str and are id are not identical at all times. We can speculate upon reason behind this phenomenon. The exactly same __creation time__ between the tweets that have the identical id_str's, might imply that the creation of these tweets happens in a very narrow time window. It could be the case for example that Twitter's servers are distributed around the world which makes it difficult to achieve complete global time synchronization between all of the databases. This could potentially make it difficult to synchronize id-stamping given that consequtive tweets might be created in a timeframe of just a few milliseconds. However the fact that there are no identical __id__-values seems to speak against this hypothesis.   

Because the actual id's are unique, we can just replace the values at id_str with the original id-values and cast them into strings. 

In [ ]:
data['id_str'] = [str(row) for row in data['id']]

In [ ]:
# Last step is to write the data to disk
data.to_json(home + '/10oct_pre_processed3.json')